In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import re

In [24]:
df = df_bak = pd.read_csv('../original_data/billboard.csv')
print(type(df))
df.head()

<class 'pandas.core.frame.DataFrame'>


,year,artist.inverted,track,time,genre,date.entered,date.peaked,x1st.week,x2nd.week,x3rd.week,...,x67th.week,x68th.week,x69th.week,x70th.week,x71st.week,x72nd.week,x73rd.week,x74th.week,x75th.week,x76th.week
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,78,63.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,15,8.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,71,48.0,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,41,23.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,57,47.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
## Uninvert artists names
df = df_bak
# print(df["artist.inverted"].head(5))
inverted_list = df["artist.inverted"].to_list()
uninverted_list = []
for artist in inverted_list:
    uninverted_artist = artist
    if "," in artist:
        artist_splitted = artist.split(",")
        uninverted_artist = artist_splitted[1] + ' ' + artist_splitted[0]
    uninverted_list.append(uninverted_artist)
df["artist.inverted"] = uninverted_list
df["artist.inverted"] = df["artist.inverted"].astype(str)
# print(df["artist.inverted"].head(5))
df = df.rename(columns={"artist.inverted": "artist"})
# df.head(5)
df_bak = df

In [26]:
## Melt weeks into one column
df = df_bak
x,y = df.shape
data_cols = df.columns[:7].to_list()
week_cols = df.columns[7:].to_list()
new_df = df.melt(data_cols, week_cols, var_name='week', value_name='rank')
df_bak = new_df

In [27]:
## Delete NaN rows
df = df_bak
new_df = df.dropna()
df_bak = new_df

In [28]:
## Get only week number
df = df_bak
old_list = df["week"].to_list()
# print(df["week"])
new_list = []
for string in old_list:
    week_number = re.findall(r'\d+', string)
    new_list.append(week_number[0])
# print(new_list)
df["week"] = new_list
df["week"] = df["week"].astype(int)
# df.head(5)
df_bak = df

In [29]:
## Change rank to Int type
df = df_bak
df["rank"] = df["rank"].convert_dtypes()
# df.head(5)
df_bak = df


In [30]:
## Shorten long track names
df = df_bak
old_list = df["track"].to_list()
new_list = []
for string in old_list:
    track_name = string
    if len(track_name) > 20:
        track_name = string[:17] + "..."
    new_list.append(track_name)
df["track"] = new_list
# df.head()
df_bak = df

In [31]:
## Create [date] column, that contains date in which certain rank was given
## Created from date.entered + week - offset (one week)
df = df_bak
df["date"] = pd.to_datetime(df["date.entered"]) + pd.to_timedelta(df["week"], unit="w") - pd.DateOffset(weeks=1)
# df.head()
df_bak = df

In [32]:
## Drop unnecessary columns [date.entered], [date.peaked] to a file
df = df_bak
side_df = df[["track", "date.entered", "date.peaked"]]
side_df = side_df.drop_duplicates(subset=["track"], keep="last")
side_df.to_csv("../analysis_data/billboard_dates.csv")
df = df.drop(columns=["date.entered", "date.peaked"])
# df.head()
df_bak = df

In [33]:
## Sort values by artist > track > week
df = df_bak
df = df.sort_values(["year", "artist", "track", "week", "rank"], ascending=True)
df.head()
df_bak = df

In [37]:
## Save data
df = df_bak
df.to_csv("../analysis_data/billboard_clean.csv")
df.head()

,year,artist,track,time,genre,week,rank,date
137,2000,Aaron Carter,Aaron's Party (Co...,3:23,R&B,1,99,2000-08-26
454,2000,Aaron Carter,Aaron's Party (Co...,3:23,R&B,2,75,2000-09-02
771,2000,Aaron Carter,Aaron's Party (Co...,3:23,R&B,3,57,2000-09-09
1088,2000,Aaron Carter,Aaron's Party (Co...,3:23,R&B,4,35,2000-09-16
1405,2000,Aaron Carter,Aaron's Party (Co...,3:23,R&B,5,35,2000-09-23
